## Drinking Detection: Masked Images
This Notebook walks through the training, validating, and testing of the CNN framework used for the masked videos

In [1]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from model import ThreeLayerConvNet
from trainer import train_loop
from data import import_data, Dataset

In [3]:
X_train, y_train, X_test, y_test, X_val, y_val = import_data(train_set = 'masked')
train_dset = Dataset(X_train, y_train, 16, shuffle = True )
val_dset = Dataset(X_test, y_test, 16, shuffle = False) ##NOTE: Validating on test per the google doc
test_dset = Dataset(X_val, y_val, 15, shuffle = False)

./data/masked/


In [4]:
def model_init_fn():
    channel_1, channel_2, num_classes = 64,64,2
    
    initializer = tf.initializers.glorot_normal()
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, strides = 1,
                                padding = "same", activation = "relu", kernel_initializer = initializer),
        tf.keras.layers.Conv2D(channel_2, 3, strides = 1,
                               padding = "same", activation = "relu", kernel_initializer = initializer),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation = "softmax", kernel_initializer = initializer )
        
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn(learning_rate):
    optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate, momentum = 0.9, nesterov = True)
    return optimizer


In [5]:
model = model_init_fn()
model.compile(optimizer_init_fn(5e-4),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=16, epochs= 5, validation_data=(X_test, y_test))

Train on 116 samples, validate on 69 samples
Epoch 1/5
116/116 [==============================] - 37s 323ms/sample - loss: 236.0724 - sparse_categorical_accuracy: 0.6897 - val_loss: 1.7003 - val_sparse_categorical_accuracy: 0.8986
Epoch 2/5
116/116 [==============================] - 31s 264ms/sample - loss: 0.5979 - sparse_categorical_accuracy: 0.9052 - val_loss: 0.1476 - val_sparse_categorical_accuracy: 0.9420
Epoch 3/5
116/116 [==============================] - 31s 264ms/sample - loss: 0.1339 - sparse_categorical_accuracy: 0.9741 - val_loss: 0.0895 - val_sparse_categorical_accuracy: 0.9565
Epoch 4/5
116/116 [==============================] - 31s 266ms/sample - loss: 0.0716 - sparse_categorical_accuracy: 0.9828 - val_loss: 0.0488 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/5
116/116 [==============================] - 31s 265ms/sample - loss: 0.0405 - sparse_categorical_accuracy: 0.9914 - val_loss: 0.0138 - val_sparse_categorical_accuracy: 1.0000


In [9]:
loss, acc = model.evaluate(X_val, y_val, batch_size = 16)
y_pred = model.predict(X_val)

47/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [10]:
y_pred = np.argmax(y_pred, axis = 1)
print (y_pred)
print (y_val.astype(np.int64))

[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0]
